# **Introduction**

***Howdy, Welcome to the Titanic***

**To whom does this notebook appeal to?**
* If you are just starting the Titanic Competition and want to learn how to implement Neural Networks, I suggest you start here!

![](https://faithmag.com/sites/default/files/styles/article_full/public/2018-09/titanic2.jpg?h=6521bd5e&itok=H8td6QVv)

**Basic Imports**

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Reading In the Data

We will be first importing the data, and creating copies. I recommend this because it give you access to a clean untouched base file.
Next, we will be dropping some things that we don't need such as passenger Id and Ticket price

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
#make a copy so your original data is not touched
train = train_data.copy()
test = test_data.copy()
train.shape
y_train = train['Survived']

#We won't need passenger ID or ticket price for the model! They do not provide much insight on the training.
Id = pd.DataFrame(test['PassengerId'])
train.drop(['PassengerId'], axis = 1, inplace=True)
test.drop(['PassengerId'], axis = 1, inplace=True)
train.drop(['Survived'], axis = 1, inplace=True)
train.drop(['Ticket'], axis = 1, inplace=True)
test.drop(['Ticket'], axis = 1, inplace=True)

**Let's take a look at our data!**

In [ ]:
train.head()

**Lets see how many null values there are! We need to fill out these values later.**

**We see that we have some midding values from age, and a ton missing from cabin**

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar='BuPu')


In [ ]:
train.isnull().sum().sort_values(ascending=False)[0:20]
# we can see that cabin is midding a lot of values, and age is tooi!

# **Data Cleaning**

**This is an awesome function I created that preprocesses the data. It does thes following**

* Fills in null values based on mean or mode
* Drops columns that are missing 50 percent of the data

*You guys are free to copy this for loop for your own projects!*[](http://)

In [ ]:


#clean the train data
for i in list(train.columns):
    dtype = train[i].dtype
    values = 0
    if(dtype == float or dtype == int):
        method = 'mean'
    else:
        method = 'mode'
    if(train[i].notnull().sum() / 891 <= .5):
        train.drop(i, axis = 1, inplace=True)
    elif method == 'mean':
        train[i]=train[i].fillna(train[i].mean())

    else:
        train[i]=train[i].fillna(train[i].mode()[0])

# WE CAN DO THIS FOR THE TEST SET TOO!

#clean the test data
for i in list(test.columns):
    dtype = test[i].dtype
    values = 0
    if(dtype == float or dtype == int):
        method = 'mean'
    else:
        method = 'mode'
    if(test[i].notnull().sum() / 418 <= .5):
        test.drop(i, axis = 1, inplace=True)
    elif method == 'mean':
        test[i]=test[i].fillna(test[i].mean())

    else:
        test[i]=test[i].fillna(test[i].mode()[0])




**As we can see, all of the missing values are gone!**

In [ ]:
sns.heatmap(train.isnull(),yticklabels=False,cbar='BuPu')


# **Feature Engineering**

**Title**

Lets take out the Mr, Miss, etc from the name section, and create a new column names title!

In [ ]:
#TITLE

train_test_data = [train, test] # combining train and test dataset

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 1, 
                 "Master": 0, "Dr": 1, "Rev": 0, "Col": 0, "Major": 0, "Mlle": 1,"Countess": 1,
                 "Ms": 1, "Lady": 1, "Jonkheer": 1, "Don": 0, "Dona" : 1, "Mme": 0,"Capt": 0,"Sir": 0 }
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    


**Sex**


We can make male and female into catagorical variables such as 0 and 1

In [ ]:
    
sex_mapping = {"male": 0, "female":1}
for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)


**Embarked**

Lets test to see if there is any correlation with Pclass and Embarked 

In [ ]:
Pclass1 = train_data[train_data['Pclass'] == 1]['Embarked'].value_counts()
Pclass2 = train_data[train_data['Pclass'] == 2]['Embarked'].value_counts()
Pclass3 = train_data[train_data['Pclass'] == 3]['Embarked'].value_counts()

df = pd.DataFrame([Pclass1, Pclass2, Pclass3])
df.index = ['1st class','2nd class', '3rd class']
df.plot(kind='bar',stacked=True, figsize=(10,5))

We can assign each embarked value to a numberical value for training later!

In [ ]:
for data in train_test_data:
    data['Embarked'] = data['Embarked'].fillna("S")
    
embarked_mapping = {"S": 0, "C": 1, "Q": 2}
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)

**Family Size**

Parch ( Parent & child ) and Sibsp( Sibling & Spouse ) are both contributing factors to family size, so lets make a new column called family size, and drop the other ones.

In [ ]:
train["FamilySize"] = train['SibSp'] + train['Parch'] + 1
test["FamilySize"] = test['SibSp'] + test['Parch'] + 1

# **Correlations**

In [ ]:
sns.heatmap(train.corr(),cbar='plasma')


In [ ]:
train.drop(['Name'], axis = 1, inplace=True)
test.drop(['Name'], axis = 1, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

# **PreProcessing**

**Now that our data looks good, lets get ready to build our models!**

In [ ]:
#imports
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D,MaxPool2D

import keras

We should scale the values in the data, so that the neural network can train better!

In [ ]:
continuous = ['Age', 'Fare', 'Parch', 'Pclass', 'SibSp', 'FamilySize']

scaler = StandardScaler()

for var in continuous:
    train[var] = train[var].astype('float64')
    train[var] = scaler.fit_transform(train[var].values.reshape(-1, 1))
for var in continuous:
    test[var] = test[var].astype('float64')
    test[var] = scaler.fit_transform(test[var].values.reshape(-1, 1))

In [ ]:
train.describe(include='all').T


# **Neural Network**

Lets create our model! make sure to add the dropout value, so that our model does not over fit.

In [ ]:
import tensorflow as tf
tf.keras.optimizers.Adam(
    learning_rate=0.003, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam', 
)
#creating a model with sequential columns
model = Sequential()

#flattens the data into a 1d array
model.add(Flatten())
#creates the first latyer with the input dimanetion. 

model.add(Dense(32, input_dim=train.shape[1],kernel_initializer = 'uniform', activation='relu'))
#next layer with 32 dense nodes
model.add(Dense(32, kernel_initializer = 'uniform', activation = 'relu'))
#drops 0.4 of the values from the next layer, so it does not over fit!

model.add(Dropout(0.4))
#last layer is initiated here
model.add(Dense(32,kernel_initializer = 'uniform', activation = 'relu'))

# create output layer
    # Feel free to experiment with the activation functions and the optimizers
model.add(Dense(1, activation='sigmoid'))  # output layer
    
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


**Let's fit our model!**

In [ ]:
history = model.fit(train, y_train, epochs=20, batch_size=50, validation_split = 0.2)

#val_acc = np.mean(training.history['val_acc'])
#print("\n%s: %.2f%%" % ('val_acc', val_acc*100))

In [ ]:
print(model.summary())


**Evaluate the model!**

In [ ]:
scores = model.evaluate(train, y_train, batch_size=32)

**Create Predictions**

In [ ]:
y_pred = model.predict(test)

y_final = (y_pred > 0.5).astype(int).reshape(test.shape[0])

output = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': y_final})
output.to_csv('prediction-ann.csv', index=False)

# **NN Analyasis**

We can plot the loss, and val_loss & acc and val_acc
The main purpose of this is to make sure the model follows a good trend, and to make sure that you are not overfitteing your model.

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,21)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,21)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

**Thank you for viewing this notebook!**

Have a great day!